In [ ]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
# load the dataset
data_2017 = pd.read_csv('data_2017.csv')

# display the column names to verify and understand the data structure
print("Columns in the dataset:")
print(data_2017.columns)


In [ ]:
# drop unnecessary columns based on your requirements
drop_cols = ['start_date_utc', 'submit_date_utc', 'network_id']
data_2017.drop(columns=[col for col in drop_cols if col in data_2017.columns], inplace=True)


In [ ]:
# fill missing values for object (string) columns only with 'Not Specified'
data_2017[data_2017.select_dtypes(include=['object']).columns] = data_2017.select_dtypes(include=['object']).fillna('Not Specified')


In [106]:
# standardize the gender column (ensure the column exists)
if 'What is your gender?' in data_2017.columns:
    data_2017['What is your gender?'] = data_2017['What is your gender?'].str.lower().replace({
        'female': 'Female', 'male': 'Male', 'f': 'Female', 'm': 'Male', 'other': 'Other'
    })
else:
    print("Gender column not found. Please verify the column name.")


In [ ]:
# convert yes/no columns to numeric where applicable
yes_no_cols = [col for col in data_2017.columns if data_2017[col].astype(str).str.contains(r'\bYes\b|\bNo\b|\bI don\'t know\b', case=False, na=False).any()]
for col in yes_no_cols:
    data_2017[col] = data_2017[col].astype(str).str.strip().replace({'Yes': 1, 'No': 0, "I don't know": -1})

In [97]:
# map categorical responses to numeric values for analysis (e.g., work interference)
interference_mapping = {"Often": 3, "Sometimes": 2, "Rarely": 1, "Never": 0}
if 'work_interfere' in data_2017.columns:
    data_2017['work_interfere'] = data_2017['work_interfere'].map(interference_mapping)


In [98]:
# save the cleaned data
data_2017.to_csv('cleaned_data_2017.csv', index=False)


In [107]:
# visualize mental health treatment by gender
if 'What is your gender?' in data_2017.columns and 'treatment' in data_2017.columns:
    plt.figure(figsize=(8, 6))
    sns.countplot(x='What is your gender?', hue='treatment', data=data_2017, palette="pastel")
    plt.title("Mental Health Treatment by Gender")
    plt.xlabel("Gender")
    plt.ylabel("Count")
    plt.legend(title="Treatment")
    plt.show()
else:
    print("Columns 'What is your gender?' or 'treatment' do not exist in the dataset.")



Columns 'What is your gender?' or 'treatment' do not exist in the dataset.


In [ ]:
# one-hot encode categorical columns for modeling
data_encoded = pd.get_dummies(data_2017, drop_first=True)



In [108]:
# define the features (X) and the target (y) - adjust column names based on your dataset
X = data_encoded[['What is your age', 'if_you_have_a_mental_health_disorder_how_often_do_you_feel_that_it_interferes_with_your_work_when_not_being_treated_effectively_ie_when_you_are_experiencing_symptoms', 'leave']]
y = data_encoded['mental_health_condition']  # adjust if 'treatment' was split into multiple columns



KeyError: "None of [Index(['What is your age',\n       'if_you_have_a_mental_health_disorder_how_often_do_you_feel_that_it_interferes_with_your_work_when_not_being_treated_effectively_ie_when_you_are_experiencing_symptoms',\n       'leave'],\n      dtype='object')] are in the [columns]"